# Exploring the RF Data Converters: a QPSK Transceiver

![](assets/transitions/lab_trans_qpsk.svg)

> In this notebook we'll introduce a Quadrature Phase Shift Keying ([QPSK](https://en.wikipedia.org/wiki/QPSK)) transceiver design as an environment to learn about the RFSoC's data converter blocks and how we can configure them using PYNQ. As shown above, this notebook touches all 3 major RFSoC domains: PS, PL, and RF. 
>
> The design from the University of Strathclyde is a full QPSK transceiver, which transmits and receives random data with full carrier and timing synchronisation. We use the PYNQ framework running on the ZCU111 board to control the design and to interactively instrument and visualize its operation.

<div style="background-color: #d9edf7; color: #31708f; padding: 15px;">
    <b>Tip:</b> Make sure you have the SMA loopback cable plugged in as per the <a href="https://github.com/Xilinx/PYNQ_RFSOC_Workshop">workshop instructions</a>.
</div>

## RF data converter refresher

The RF data converters are the RFSoC's means of communicating to and from the analog world.
The ZCU111 board gives us:
  * 8 Digital to Analog Converter (DAC) blocks
      + Sampling up to 6.5 GSPS
      + Grouped into 2 independent tiles
  * 8 Analog to Digital Converter (ADC) blocks
      + Sampling up to 4 GSPS
      + Grouped into 4 independent tiles

These blocks include more than just the RF DAC/ADCs. Each one also contains a gearbox FIFO, up to 8$\times$ interpolator/decimator, a complex mixer, and Quadrature Modulation Correction (QMC) unit. (The QMC is used to offset potetial imbalance in the quality of the I/Q signals). Here is a simplified version of an ADC block datapath:

![](assets/adc_datapath.svg)

We'll be using 1 DAC block and 1 ADC block in this design and will touch some of the configuration options in both.

## Import libraries

Start by including the `xrfdc` drivers so we can configure the RF data converters, `ipywidgets` to make interactive controls, `numpy` for numerical analysis. We also import a module (`rfsoc_qpsk`) to control our QPSK design — we're abstracting some of the nuances of the Python code to let us focus on the data converters just now. We'll dive into the software details [later](./04_dsp_and_python.ipynb) in the session.

In [ ]:
import xrfdc
import ipywidgets as ipw
import numpy as np

from rfsoc_qpsk.qpsk_overlay import QpskOverlay

## Download the QPSK bitstream

Let's download the bitstream and tell it not to initialize the RF clocks (the workshop SD card already does this on boot).

In [ ]:
ol = QpskOverlay(init_rf_clks=False)

For a quick reference of all the things you can do with the QPSK overlay, ask the Python interpreter!
Pop open a new console (right click here and select "_New Console for Notebook_") and type `ol.plot_group?` to query a method of our new overlay. Press the <_Shift Enter_> keys to execute the query.  Close the new console window when you're finished with it.

## Inspecting the transmit path

There are 3 main steps in the QPSK transmit IP signal path:

1. Random symbol generation
2. Pulse shaping
3. Interpolation
  
This design "taps off" this path at several points so we can inspect the signals in JupyterLab.
The RF data converter can be reconfigured from Python too - we'll look at that [later](#Reconfigure-the-RF-Data-Path).

![](./assets/block_diagram/QPSK_system_block_diagrams_Tx_only.svg)

First we plot our raw QPSK symbols in the time domain. If no plot appears, check that you are using the JupyterLab interface (`192.168.2.99/lab`).

In [ ]:
ol.plot_group(
  'tx_symbol',            # Plot group ID
  ['time-binary'],        # List of plot types chosen from:
                          #   ['time','time-binary','frequency','constellation']
  ol.qpsk_tx.get_symbols, # Function to grab a buffer of samples
  500                     # Sample frequency (Hz)
)

We can stream new samples into this plot using the play/stop buttons. By default the samples are stored in a rolling buffer so we can keep this running for a while without worrying too much about total memory usage. As you continue to work through this notebook though, you should stop any previous plot streams for best browser performance.

The original signals are filtered using a root raised cosine (RRC) filter to produce "pulse-shaped" signals, which have better properties for transmission.  Let's have a look at the pulse-shaped signals in both the time and frequency domains. The FFT, that is used in calculating the frequency response, is accelerated in the PL so we pass in an extra argument, `get_freq_data`, telling the plotting library how to grab the accelerated FFT data.

In [ ]:
ol.plot_group('tx_shaped', ['time', 'frequency'], ol.qpsk_tx.get_shaped_time, 4000,
              get_freq_data=ol.qpsk_tx.get_shaped_fft)

## Inspecting the receive path

The receive side is nearly the inverse of the transmit path (there's just some extra work for properly synchronising).

Again, there are taps off from a few different places in the signal path:

![](./assets/block_diagram/QPSK_system_block_diagrams_Rx_only.svg)

Because there are a few different intermediate stages, let's reuse the same cells to plot any of them on-demand.

First we describe how to generate plots for each of the intermediate taps.

In [ ]:
rx_domains = ['time', 'frequency', 'constellation']

plot_rx_decimated   = lambda : ol.plot_group(
    'rx_decimated',   rx_domains, ol.qpsk_rx.get_decimated,     1000
)

# Note 'rx_rrced' is the signal at the output of the matched filter, which performs a root raised cosine transform
plot_rx_rrced       = lambda : ol.plot_group(
    'rx_rrced',       rx_domains, ol.qpsk_rx.get_rrced,         16000
)

plot_rx_coarse_sync = lambda : ol.plot_group(
    'rx_coarse_sync', rx_domains, ol.qpsk_rx.get_coarse_synced, 4000
)

Now we can just execute the function for whichever tap you want. For example, let's look at the tap after decimation below.

In [ ]:
plot_rx_decimated()

And for the final plot, let's look at the synchronised output data. To recover the bits we need to take our sampled, synchronised signal (seen in the constellation plot below) and decide which quadrant each symbol has fallen into.

In [ ]:
def classify_bits(frame):
    bit_quantise    = lambda b: 1 if b>0 else 0
    symbol_quantise = lambda i, q: bit_quantise(i) + 1j*bit_quantise(q)
    return np.fromiter(
        map(symbol_quantise, np.real(frame), np.imag(frame)),
        dtype=np.complex
    )

ol.plot_group(
    'rx_data',
    ['constellation', 'time-binary'],
    lambda : classify_bits(ol.qpsk_rx.get_data()),
    500,
    get_const_data=ol.qpsk_rx.get_data
)

Next, we configure some RF settings and demonstrate their impact on the operation of the QPSK transceiver.  Now is a good time to exploit how JupyterLab can handle multiple windows.

You should create a new window for the constellation plot and leave it streaming. Make the new window by right clicking the current plot and selecting "_Create New View for Output_". 

Position the output window so that you can see both it and the notebook window at the same time.  To do this, snap the new window to the side by clicking the window's title ("Output View") and dragging it to the side of the web page. 

Now you will be able to configure different RF settings (in the following cells) while still getting instant feedback about the received signal — pretty neat!

## Reconfigure the RF Data Path

### Transmit Power

The QPSK design includes a digital attenuator on the transmit path. We can configure this via a memory-mapped register.

Let's use this as an example of interactive reconfiguration because the effects are quite clear in the constellation diagram. Try reducing the output power by setting a gain between 0 (off) and 1 (full scale).

In [ ]:
ol.qpsk_tx.set_gain(0.6)

The constellation plot should shrink in a little towards the origin. Let's return to full power now.

In [ ]:
ol.qpsk_tx.set_gain(1)

We can use some `ipywidgets` to make a more natural interface to control the gain too. Let's expose this as a slider with a callback to the `set_gain` function.

In [ ]:
pow_slider = ipw.SelectionSlider(
    options=[0.1, 0.3, 0.6, 1],
    value=1,
    description='',
)

accordion = ipw.Accordion(children=[pow_slider])
accordion.set_title(0, 'Transmitter power')
display(accordion)

def unwrap_slider_val(callback):
    return lambda slider_val : callback(slider_val['new'])

pow_slider.observe(unwrap_slider_val(ol.qpsk_tx.set_gain), names='value')

### Transmit and Receive Mixer Settings

So far the RF Data Converter settings have been controlled by `QpskOverlay` but we can also reconfigure these on the fly in python with the `xrfdc` driver. This driver is a Python wrapper around the existing [baremetal driver](https://github.com/Xilinx/embeddedsw/tree/release-2018.3/XilinxProcessorIPLib/drivers/rfdc).

First of all, consider a simplified version of the DAC block used in the transmit side.

![](./assets/block_diagram/RF_DAC.svg)

There's a lot of scope for reconfiguration here. We've annotated just a couple of settings with their corresponding function names, but see the [IP product guide](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_1/pg269-rf-data-converter.pdf) for more details.

As an example, let's play with the mixer settings. Try changing the DAC's mixer frequency from our default 1000 MHz to 900 MHz.

In [ ]:
def update_nco(rf_block, nco_freq):
    rf_block.MixerSettings['Freq'] = nco_freq
    rf_block.UpdateEvent(xrfdc.EVENT_MIXER)

update_nco(ol.dac_block, 900)

The received signal should disappear until we configure the receiver's ADC to match the new carrier frequency. Set the new carrier frequency for the ADC side mixer below.

In [ ]:
update_nco(ol.adc_block, 900)

Again, we can use `ipywidgets` to make an interactive interface for these settings. Below we setup an RX and a TX slider and a TX slider that are linked together so we can scrub along the spectrum keeping both sides in near lock-step. If you've got any analog RF filters to hand, try them out with different mixer settings!

In [ ]:
def new_nco_slider(title):
    return ipw.FloatSlider(
        value=1000,
        min=620,
        max=1220,
        step=20,
        description=title,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
    )

tx_nco_slider = new_nco_slider('TX (MHz)')
rx_nco_slider = new_nco_slider('RX (MHz)')

accordion = ipw.Accordion(children=[ipw.VBox([tx_nco_slider, rx_nco_slider])])
accordion.set_title(0, 'Carrier frequency')
display(accordion)

ipw.link((rx_nco_slider, 'value'), (tx_nco_slider, 'value'))
tx_nco_slider.observe(
    unwrap_slider_val(lambda v: update_nco(ol.dac_block, v)),
    names='value'
)
rx_nco_slider.observe(
    unwrap_slider_val(lambda v: update_nco(ol.adc_block, v)),
    names='value'
)

If you've got time, do feel free to try out any RF data converter settings you like. For example, the ADC's `QMCSettings` could be used to adjust the gain and phase at the receive side.

In [ ]:
ol.adc_block.QMCSettings = {
    'EnableGain': 1,
    'EnablePhase': 1,
    'EventSource': xrfdc.EVNT_SRC_TILE,
    'GainCorrectionFactor': 0.5,
    'OffsetCorrectionFactor': 0,
    'PhaseCorrectionFactor': 25.0
}
ol.adc_block.UpdateEvent(xrfdc.EVENT_QMC)

And finally, let's ensure that all of the streaming plots are shut down before moving on. We don't want to be polling for new plot data after loading a different bitstream! 

In [ ]:
for thread_group in ol.timers.registry.values():
    for thread in thread_group:
        thread.stop()

## Summary

We've now lead you through how we can interact with the RF data converters from PYNQ, using a QPSK transmit/receive loopback system as an example. In full, we've shown:

  * Use of the programmable logic in the context of a real RF application
  * Performing on-board introspection of an RF design:
      * Dashboard style visualisation
      * Leveraging existing plotting libraries from the Python ecosystem
  * Interacting with a QPSK hardware design
      * Configuring the signal path, using transmit power as an example
      * Configuring the RF data converter, using TX/RX mixer frequencies as an example

Another key takeaway is that this design has been open sourced. Feel free to take a look at our [repo](https://github.com/strath-sdr/rfsoc_qpsk) to see the hardware design or the supporting Python code.

Now that we've briefly explored the data converter blocks, let's take a look at the other unique capability of the RFSoC — the SD-FEC blocks.

[⬅️ Previous](01_introduction.ipynb) 👩‍💻  [Next ➡️](03_pynq_and_sd_fec.ipynb)